## knowlege base construction 
### docuemnts-->chunk-->embedding-->vector database
- if use deeprec to deal with docuemts, maybe you need do next:
- RAG has its own requirement.txt, while deepdoc has another requirements: pyproject.toml  in ragflaw project, run `pip install -e .` 


In [14]:
import sys
print(sys.version)

3.10.18 (main, Jun  5 2025, 08:37:47) [Clang 14.0.6 ]


In [ ]:
import model
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')   
nltk.download('wordnet')
nltk.download('omw-1.4')   

[nltk_data] Downloading package punkt to /Users/tomli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/tomli/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tomli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/tomli/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="BAAI/bge-m3",
    local_dir="./data/llm_app/embedding_models/bge-m3"
)

In [17]:
from model import RagEmbedding, RagLLM
from doc_parse import chunk, read_and_process_excel, logger
# RAG has its own requirement.txt, while deepdoc has another requirements: pyproject.toml  in ragflaw project, run `pip install -e .` wh


In [18]:
import pandas as pd
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb

In [19]:
pdf_files = ['./data/zhidu_employee.pdf',
            './data/zhidu_travel.pdf']

excel_files = ['./data/zhidu_detail.xlsx']

In [20]:
r_spliter = RecursiveCharacterTextSplitter(chunk_size=128,
                                          chunk_overlap=30,
                                          separators=["\n\n",
                                                      "\n", 
                                                      ".",
                                                      "\uff0e",  # Fullwidth full stop
                                                       "\u3002",  # Ideographic full stop
                                                      ",",
                                                      "\uff0c",  # Fullwidth comma
                                                      "\u3001",  # Ideographic comma
                                                     ])

In [21]:
doc_data = []
for pdf_file_name in pdf_files:
    res = chunk(pdf_file_name, callback=logger)
    for data in res:
        content = data["content_with_weight"]
        if '<table>' not in content and len(content) > 200:
            doc_data = doc_data + r_spliter.split_text(content)
        else:
            doc_data.append(content)

OCR is running...


OCR finished.
OCR: 0.7383865410010912
Layout analysis finished.
layouts: 4.1743157910023
Table analysis finished.
Text merging finished
OCR is running...


OCR finished.
OCR: 0.44644279099884443
Layout analysis finished.
layouts: 3.027153791001183
Table analysis finished.
Text merging finished


In [22]:
for i in doc_data:
    print(len(i), "="*10,  i)

522 ========== <table><caption>病假发放标准：</caption>
<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>
<tr><td></td><td  >不满二年</td><td  >60% </td></tr>
<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>
<tr><td  >6 个月以内病假</td><td  >已满四年不满六年</td><td  >80% </td></tr>
<tr><td></td><td  >已满六年不满八年</td><td  >90% </td></tr>
<tr><td></td><td  >八年以上</td><td  >100% </td></tr>
<tr><td  rowspan=3 >6 个月以上病假</td><td  >不满一年</td><td  >40% </td></tr>
<tr><td  >已满一年不满三年</td><td  >50% </td></tr>
<tr><td  >连续工龄三年以上</td><td  >60% </td></tr>
</table>
41 ========== 教职工考勤管理制度
第一节适用范围
1、本制度包括了考勤、休假、加班等方面的规定。
64 ========== 2、本制度适用于学校全体教职员工。
第二节考勤规定
1、学校的工作时间由学校决定并公布。学校内除特殊岗位特别规定外，全体教职员工
66 ========== 均应严格执行学校的作息时间表，不迟到、不早退、不中途离校。
工作时间：星期一至星期四7:55-16:55 星期五7:55-16:15
44 ========== 2、所有教职工实行考勤打卡制度，工作日内，每天需打卡两次，上午上班一次和下午
下班一次。
113 ========== 3、教职员工因故(特殊情况除外)晚到或早退，应事先履行请假手续，经批准后方可离
校。原则上，员工请假无论时间长短、假期形式，除急诊病假或突发事件外，一律需按照请
假流程。
请假，需事先在钉钉系统中提交申请。
有效的请假流程为：
106 ========== 请假，需事先在钉钉系统中提交申请。
有效的请假流程为：
（1）员工休假必须事先向部门负

In [23]:
for excel_file_name in excel_files:
    data = read_and_process_excel(excel_file_name)
    df = pd.DataFrame(data[8:], columns=data[7])
    data_excel = df.drop(columns=df.columns[11:17])
    content = data_excel.to_markdown(index=False).replace(' ', "")
    doc_data.append(f"### 以下是中央和国家机关工作人员赴地方差旅住宿费标准明细表： \n\n {content}")

In [24]:
from langchain_core.documents import Document
import uuid
documents = []
document_ids = []
for idx, chunk in enumerate(doc_data):
    is_table = 0
    if "table" in chunk:
        is_table = 1
    if idx == len(doc_data) - 1:
        is_table = 1
    doc_id = str(uuid.uuid4())
    document = Document(
        page_content=chunk,
        metadata={"type": "ori", "is_table": is_table})
    documents.append(document)
    document_ids.append(doc_id)

In [25]:
embedding_cls = RagEmbedding()

/Users/tomli/Desktop/tmp/model.py:62: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedding = HuggingFaceEmbeddings(model_name=model_path,


In [26]:
doc_txts = dict(zip(document_ids, documents))

In [27]:
import pickle
with open('./data/zhidu_db.pickl', 'wb') as file:
    pickle.dump(doc_txts, file)

In [28]:
chroma_client = chromadb.HttpClient(host="localhost", port=8000)

In [29]:
# small sample to see if it works on Chroma
embedding_db = Chroma.from_documents(
    documents[:10],
    embedding_cls.get_embedding_fun(),
    client=chroma_client,
    collection_name="zhidu_db",
)



In [36]:
# chroma_client.delete_collection("zhidu_db")
embedding_db = Chroma.from_documents(documents,
                                     embedding_cls.get_embedding_fun(),
                                     client=chroma_client,
                                     collection_name="zhidu_db",
                                     )

In [38]:
query = '迟到有什么规定？'

In [39]:
related_docs = embedding_db.similarity_search(query, k=2)

In [40]:
related_docs

[Document(metadata={'is_table': 0, 'type': 'ori'}, page_content='4、无工作理由，超过上班时间到岗的，视为迟到；未到下班时间提前离校的，视为早\n退；中途未经批准离校，视为旷工；迟到、早退、旷工者按照相关办法处理。'),
 Document(metadata={'is_table': 0, 'type': 'ori'}, page_content='1. 迟到、早退\n(1)劳动考勤是公司支付薪资的依据和职工年度岗位考核内容之一。（2）迟到或早退60\n分钟以上（含60 分钟），每次视同缺勤1 天。（3）职工迟到、早退、脱岗累计超过3 次的（含），\n从第1 次起，每次扣减工资50 元。')]

In [ ]:
# zhidu_db = Chroma("zhidu_db", 
#                   embedding_model.get_embedding_fun(), 
#                   client=chroma_client)
# zhidu_db.similarity_search(query, k=k)

In [ ]:

# chroma_client = chromadb.HttpClient(host="localhost", port=8000)

# def delete_chroma_collection(name, embedding_fn, chroma_client):
#     collection = Chroma(name,
#                         embedding_fn,
#                         client=chroma_client)
#     Chroma.delete_collection(collection)

In [ ]:
# delete_chroma_collection("zhidu_db", embedding_cls.get_embedding_fun(), chroma_client)